# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperparameter-tuning-experiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy =  BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
                                            '--solver': choice('newton-cg', 'lbfgs', 'liblinear'),
                                            #'--penalty': choice('none', 'l1', 'l2'),
                                            '--C': choice(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100),
                                            '--max_iter': choice(100, 200, 300, 400, 500)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig( hyperparameter_sampling=ps, 
                                        primary_metric_name='Accuracy', 
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                        max_total_runs=100, 
                                        max_concurrent_runs=4, 
                                        max_duration_minutes=10080, 
                                        policy=early_termination_policy, 
                                        estimator=estimator
)

In [ ]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run_config).show()
assert(hyperdrive_run.get_status() == 'Completed')

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best parameters: ', parameter_values)
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


In [ ]:
#TODO: Save the best model

best_model = best_run.register_model(model_name='udacity_project_model', model_path='./')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service